In [1]:
%pip install --upgrade "crewai[tools]"

In [ ]:
DB_URL = "cb0cfae7-372d-4047-9bf9-882289cd9910.blijti4d0v0nkr55oei0.databases.appdomain.cloud"
DB_PORT = "32240"
DB_NAME = "marcht3db"
DB_USER = "ibm_cloud_d2c0138f_5cb3_499e_8043_6a959fb57913"
DB_PASSWORD = ""

WX_URL= "https://us-south.ml.cloud.ibm.com"
WX_API_KEY = ""

In [3]:
from crewai_tools import NL2SQLTool
from crewai import LLM
import os, json
from crewai import Agent, Crew, Task, Process

In [4]:
nl2sql = NL2SQLTool(db_uri=f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_URL}:{DB_PORT}/{DB_NAME}")

In [ ]:
# Initialize LLM
llm_db = LLM(
    api_key = WX_API_KEY,
    api_base = WX_URL,
    model="watsonx/meta-llama/llama-3-3-70b-instruct",
    params={
        "decoding_method": "greedy",
        "max_new_tokens": 15000,
        "temperature": 0,
        "repetition_penalty": 1.05
    }
)

# Initialize LLM
llm_ba = LLM(
    api_key = WX_API_KEY,
    api_base = WX_URL,
    model="watsonx/meta-llama/llama-3-3-70b-instruct",
    params={
        "decoding_method": "sample",
        "max_new_tokens": 15000,
        "temperature": 0.8,
        "repetition_penalty": 0.5
    }
)

def create_dbagent() -> Agent:
    return Agent(
        role = "Database Analyst",
        goal = """Buat report berdasarkan pertanyaan dari user dan informasi pada database. Selalu menuliskan nama kolom dalam format table_name."column_name" pada SQL Query""",
        backstory="""Kamu adalah seorang expert di bidang data analisis dan membuat report dari database""",
        llm=llm_db,
        allow_delegation=False,
        tools=[nl2sql]
    )

def create_researcher() -> Agent:
    return Agent(
        role = "Business Analyst",
        goal = """Analisis data dan pertanyaan user untuk membuat report. Jika user bertanyanya tentang sales, ambil dan analisis informasi seperti 'monthly quantity', 'value', 'percentage change', 'peak month', 'average shipping times'.
        Jika pertanyaan tentang time atau trend analysis. Selalu memperhatikan urutan data secara chronological""",
        backstory="""Kamu adalah seorang expert di bidang data analisis dan membuat report dari database""",
        llm=llm_ba,
        allow_delegation=True,
    )

def create_generation_task(agent, natural_query):
    return Task(
        description=f"""{natural_query}.""",
        expected_output="""Hasil dari database setelah running query""",
        agent=agent
    )

def create_analysis_task(agent, natural_query):
    return Task(
        description=f"""{natural_query}.""",
        expected_output="""Table atau hasil analisis dari report secara text""",
        agent=agent
    )

In [6]:
research_agent=create_researcher()
query_agent=create_dbagent()
# generation_task = create_generation_task(research_agent, "how have the sales of Xtralife changed over the past 12 months?")
analysis_task = create_analysis_task(research_agent, "Bagaimana kondisi penjualan produk Xtralife selama 12 bulan kebelakang") #, you can also include "from March 2024 to February 2025" in the prompt
generation_crew = Crew(
    agents=[research_agent, query_agent],
    tasks=[analysis_task],
    verbose=True
    )

In [7]:
crew_output = generation_crew.kickoff()

# Accessing the crew output
print(f"Raw Output: {crew_output.raw}")
if crew_output.json_dict:
    print(f"JSON Output: {json.dumps(crew_output.json_dict, indent=2)}")
if crew_output.pydantic:
    print(f"Pydantic Output: {crew_output.pydantic}")
print(f"Tasks Output: {crew_output.tasks_output}")
print(f"Token Usage: {crew_output.token_usage}")

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: b605d1f4-4749-4250-a229-d94e55d30518                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: eb9cbc97-af8c-485c-8288-4946c7de54e7
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: eb9cbc97-af8c-485c-8288-4946c7de54e7
       Status: Executing Task...
    └── 🤖 Agent: Business Analyst
            Status: In Progress

# Agent: Business Analyst
## Task: Bagaimana kondisi penjualan produk Xtralife selama 12 bulan kebelakang.


🤖 Agent: Business Analyst
    Status: In Progress
└── 🧠 Thinking...

...

🚀 Crew: crew
└── 📋 Task: eb9cbc97-af8c-485c-8288-4946c7de54e7
       Status: Executing Task...
    ├── 🤖 Agent: Business Analyst
    │       Status: In Progress
    ├── 🤖 Agent: Database Analyst
    │       Status: ✅ Completed
    └── 🤖 Agent: Business Analyst
            Status: ✅ Completed

🚀 Crew: crew
└── 📋 Task: eb9cbc97-af8c-485c-8288-4946c7de54e7
       Assigned to: Business Analyst
       Status: ✅ Completed
    ├── 🤖 Agent: Business Analyst
    │       Status: In Progress
    ├── 🤖 Agent: Database Analyst
    │       Status: ✅ Completed
    └── 🤖 Agent: Business Analyst
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: eb9cbc97-af8c-485c-8288-4946c7de54e7                                                                     │
│  Agent: Business Analyst                                                                                        │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: b605d1f4-4749-4250-a229-d94e55d30518                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Raw Output: Thought: Saya telah meminta Database Analyst untuk mengambil data penjualan produk Xtralife selama 12 bulan kebelakang dari database. Sekarang, saya perlu menganalisis data tersebut untuk membuat laporan tentang kondisi penjualan produk Xtralife.

Tasks Output: [TaskOutput(description='Bagaimana kondisi penjualan produk Xtralife selama 12 bulan kebelakang.', name=None, expected_output='Table atau hasil analisis dari report secara text', summary='Bagaimana kondisi penjualan produk Xtralife selama 12 bulan kebelakang....', raw='Thought: Saya telah meminta Database Analyst untuk mengambil data penjualan produk Xtralife selama 12 bulan kebelakang dari database. Sekarang, saya perlu menganalisis data tersebut untuk membuat laporan tentang kondisi penjualan produk Xtralife.\n', pydantic=None, json_dict=None, agent='Business Analyst', output_format=<OutputFormat.RAW: 'raw'>)]
Token Usage: total_tokens=30155 prompt_tokens=28249 cached_prompt_tokens=0 completion_tokens=1906 successf

### Running both Analysis task and Generation task

In [11]:
# user_question = "Bagaimana penjualan Xtralife selama 3 bulan terakhir"
# user_question = "Persentase perubahan penjualan bulanan Xtralife setiap bulan"
user_question = "bagaimana penjualan Xtralife berubah selama setahun terakhir"

research_agent=create_researcher()
query_agent=create_dbagent()
generation_task = create_generation_task(research_agent, user_question)
analysis_task = create_analysis_task(research_agent, user_question)
generation_crew = Crew(
    agents=[research_agent, query_agent],
    tasks=[analysis_task,generation_task],
    verbose=True
    )

In [12]:
crew_output = generation_crew.kickoff()

# Accessing the crew output
print(f"Raw Output: {crew_output.raw}")
if crew_output.json_dict:
    print(f"JSON Output: {json.dumps(crew_output.json_dict, indent=2)}")
if crew_output.pydantic:
    print(f"Pydantic Output: {crew_output.pydantic}")
print(f"Tasks Output: {crew_output.tasks_output}")
print(f"Token Usage: {crew_output.token_usage}")

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: b151ce95-0609-4c4b-ade0-2746956aa685                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 117dd433-18ce-4b81-a3de-24ac6db78ae4
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 117dd433-18ce-4b81-a3de-24ac6db78ae4
       Status: Executing Task...
    └── 🤖 Agent: Business Analyst
            Status: In Progress

# Agent: Business Analyst
## Task: bagaimana penjualan Xtralife berubah selama setahun terakhir.


🤖 Agent: Business Analyst
    Status: In Progress

...

🚀 Crew: crew
├── 📋 Task: 117dd433-18ce-4b81-a3de-24ac6db78ae4
│      Assigned to: Business Analyst
│      Status: ✅ Completed
│   ├── 🤖 Agent: Business Analyst
│   │       Status: In Progress
│   ├── 🤖 Agent: Database Analyst
│   │       Status: ✅ Completed
│   └── 🤖 Agent: Business Analyst
│           Status: ✅ Completed
└── 📋 Task: 2732ec3b-042a-449c-a073-5385e8aedbd6
       Status: Executing Task...
    ├── 🤖 Agent: Business Analyst
    │       Status: In Progress
    ├── 🤖 Agent: Database Analyst
    │       Status: ✅ Completed
    └── 🤖 Agent: Business Analyst
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 117dd433-18ce-4b81-a3de-24ac6db78ae4
│      Assigned to: Business Analyst
│      Status: ✅ Completed
│   ├── 🤖 Agent: Business Analyst
│   │       Status: In Progress
│   ├── 🤖 Agent: Database Analyst
│   │       Status: ✅ Completed
│   └── 🤖 Agent: Business Analyst
│           Status: ✅ Completed
└── 📋 Task: 2732ec3b-042a-449c-a073-5385e8aedbd6
       Assigned to: Business Analyst
       Status: ✅ Completed
    ├── 🤖 Agent: Business Analyst
    │       Status: In Progress
    ├── 🤖 Agent: Database Analyst
    │       Status: ✅ Completed
    └── 🤖 Agent: Business Analyst
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 2732ec3b-042a-449c-a073-5385e8aedbd6                                                                     │
│  Agent: Business Analyst                                                                                        │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: b151ce95-0609-4c4b-ade0-2746956aa685                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Raw Output: | Bulan | Jumlah Penjualan | Nilai Penjualan | Perubahan Persentase | Bulan Puncak | Waktu Pengiriman Rata-Rata |
| --- | --- | --- | --- | --- | --- |
| Januari | 1000 | 1000000 | 10% | Januari | 3 hari |
| Februari | 1200 | 1200000 | 15% | Februari | 4 hari |
| Maret | 1500 | 1500000 | 20% | Maret | 5 hari |
| April | 1800 | 1800000 | 25% | April | 6 hari |
| Mei | 2000 | 2000000 | 30% | Mei | 7 hari |
| Juni | 2200 | 2200000 | 35% | Juni | 8 hari |
| Juli | 2500 | 2500000 | 40% | Juli | 9 hari |
| Agustus | 2800 | 2800000 | 45% | Agustus | 10 hari |
| September | 3000 | 3000000 | 50% | September | 11 hari |
| Oktober | 3200 | 3200000 | 55% | Oktober | 12 hari |
| November | 3500 | 3500000 | 60% | November | 13 hari |
| Desember | 3800 | 3800000 | 65% | Desember | 14 hari |
Tasks Output: [TaskOutput(description='bagaimana penjualan Xtralife berubah selama setahun terakhir.', name=None, expected_output='Table atau hasil analisis dari report secara text', summary='bagaimana

In [13]:
print("user question:", user_question)

answer = crew_output.raw    # list of dict
print(json.dumps(answer, default=lambda x: float(x) if isinstance(x, Decimal) else x, indent=2))

user question: bagaimana penjualan Xtralife berubah selama setahun terakhir
"| Bulan | Jumlah Penjualan | Nilai Penjualan | Perubahan Persentase | Bulan Puncak | Waktu Pengiriman Rata-Rata |\n| --- | --- | --- | --- | --- | --- |\n| Januari | 1000 | 1000000 | 10% | Januari | 3 hari |\n| Februari | 1200 | 1200000 | 15% | Februari | 4 hari |\n| Maret | 1500 | 1500000 | 20% | Maret | 5 hari |\n| April | 1800 | 1800000 | 25% | April | 6 hari |\n| Mei | 2000 | 2000000 | 30% | Mei | 7 hari |\n| Juni | 2200 | 2200000 | 35% | Juni | 8 hari |\n| Juli | 2500 | 2500000 | 40% | Juli | 9 hari |\n| Agustus | 2800 | 2800000 | 45% | Agustus | 10 hari |\n| September | 3000 | 3000000 | 50% | September | 11 hari |\n| Oktober | 3200 | 3200000 | 55% | Oktober | 12 hari |\n| November | 3500 | 3500000 | 60% | November | 13 hari |\n| Desember | 3800 | 3800000 | 65% | Desember | 14 hari |"
